In [6]:
import os
import numpy as np
import h5py
import random
from PIL import Image
import tensorflow as tf
import json

#from meta import Meta




In [104]:
class ExampleReader(object):
    def __init__(self, path_to_image_files):
        self._path_to_image_files = path_to_image_files
        self._num_examples = len(self._path_to_image_files)
        self._example_pointer = 0


    def _get_attrs(digit_struct_mat_file, index):
        """
        Returns a dictionary which contains keys: label, left, top, width and height, each key has multiple values.
        """
        attrs = {}
        f = digit_struct_mat_file
        item = f['digitStruct']['bbox'][index].item()
        for key in ['label', 'left', 'top', 'width', 'height']:
            attr = f[item][key]
            values = [f[attr.value[i].item()].value[0][0]
                      for i in range(len(attr))] if len(attr) > 1 else [attr.value[0][0]]
            attrs[key] = values
            
        print(attrs)
        return attrs


    def _preprocess(image, bbox_left, bbox_top, bbox_width, bbox_height):
        cropped_left, cropped_top, cropped_width, cropped_height = (int(round(bbox_left - 0.15 * bbox_width)),
                                                                    int(round(bbox_top - 0.15 * bbox_height)),
                                                                    int(round(bbox_width * 1.3)),
                                                                    int(round(bbox_height * 1.3)))
        image = image.crop([cropped_left, cropped_top, cropped_left + cropped_width, cropped_top + cropped_height])
        image = image.resize([64, 64])
#         image = image.resize([128, 128])
        return image
    
    def _int64_feature(value):
        return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

    def _float_feature(value):
        return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
    
    def _bytes_feature(value):
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

    def read_and_convert(self, digit_struct_mat_file):
        """
        Read and convert to example, returns None if no data is available.
        """
        if self._example_pointer == self._num_examples:
            return None
        path_to_image_file = self._path_to_image_files[self._example_pointer]
        print(path_to_image_file)
        
        index = int(path_to_image_file.split('/')[-1].split('.')[0]) - 1
        self._example_pointer += 1

        attrs = ExampleReader._get_attrs(digit_struct_mat_file, index)
        label_of_digits = attrs['label']
        length = len(label_of_digits)
        if length > 5:
            # skip this example
            return self.read_and_convert(digit_struct_mat_file)

        digits = [10, 10, 10, 10, 10]   # digit 10 represents no digit
        for idx, label_of_digit in enumerate(label_of_digits):
            digits[idx] = int(label_of_digit if label_of_digit != 10 else 0)    # label 10 is essentially digit zero

        attrs_left, attrs_top, attrs_width, attrs_height = map(lambda x: [int(i) for i in x], [attrs['left'], attrs['top'], attrs['width'], attrs['height']])
        min_left, min_top, max_right, max_bottom = (min(attrs_left),
                                                    min(attrs_top),
                                                    max(map(lambda x, y: x + y, attrs_left, attrs_width)),
                                                    max(map(lambda x, y: x + y, attrs_top, attrs_height)))
        center_x, center_y, max_side = ((min_left + max_right) / 2.0,
                                        (min_top + max_bottom) / 2.0,
                                        max(max_right - min_left, max_bottom - min_top))
        bbox_left, bbox_top, bbox_width, bbox_height = (center_x - max_side / 2.0,
                                                        center_y - max_side / 2.0,
                                                        max_side,
                                                        max_side)
        image = np.array(ExampleReader._preprocess(Image.open(path_to_image_file), bbox_left, bbox_top, bbox_width, bbox_height))

#         print(image.shape)
#         img = Image.fromarray(image, 'RGB')
#         img.show()
        image = image.tobytes()
        
        example = tf.train.Example(features=tf.train.Features(feature={
            'image': ExampleReader._bytes_feature(image),
            'length': ExampleReader._int64_feature(length),
            'digits': tf.train.Feature(int64_list=tf.train.Int64List(value=digits))
        }))
        return example
    
    
    def read_and_convert_sportfoto(self, image_info_df):
        
        if self._example_pointer == self._num_examples:
            return None
    
        path_to_image_file = self._path_to_image_files[self._example_pointer]

        image_index = int(path_to_image_file.split('/')[-1].split('.')[0]) 
        self._example_pointer += 1

        row = image_info_df.loc[image_index]
        bbox_left, bbox_top, bbox_width, bbox_height = ( row['left'], row['top'],
                                                        row['right'] - row['left'],
                                                        row['bottom'] - row['top'])
           
        digits = [10,10,10,10,10]
        length = len(str(row['numbers']))

        for ix,c in enumerate(str(row['numbers'])):
            digits[ix] = int(c)
   
        image = np.array(ExampleReader._preprocess(Image.open(path_to_image_file), bbox_left, bbox_top, bbox_width, bbox_height))
        
        image = image.tobytes()
        
#         image = Image.fromarray(image, 'RGB')
#         image.show()

        example = tf.train.Example(features=tf.train.Features(feature={
            'image': ExampleReader._bytes_feature(image),
            'length': ExampleReader._int64_feature(length),
            'digits': tf.train.Feature(int64_list=tf.train.Int64List(value=digits))
        }))
        return example

In [106]:
import pandas as pd

path_to_train_dir = './toy_data/train/'

path_to_image_files = tf.gfile.Glob(os.path.join(path_to_train_dir, '*.jpg'))

er = ExampleReader(path_to_image_files)
csv_path = "/home/egert/Desktop/Number_reader_from_sportfoto/images_data.csv"
data = pd.read_csv(csv_path, index_col='image')

# er._example_pointer = 25

# er.read_and_convert_sportfoto(data)

In [109]:
def convert_to_tfrecords(path_to_dataset_dir_and_digit_struct_mat_file_tuples,
                         path_to_tfrecords_files, choose_writer_callback):
    num_examples = []
    writers = []

    for path_to_tfrecords_file in path_to_tfrecords_files:
        num_examples.append(0)
        writers.append(tf.python_io.TFRecordWriter(path_to_tfrecords_file))

    for path_to_dataset_dir, path_to_digit_struct_mat_file in path_to_dataset_dir_and_digit_struct_mat_file_tuples:
#         path_to_image_files = tf.gfile.Glob(os.path.join(path_to_dataset_dir, '*.png'))
        path_to_image_files = tf.gfile.Glob(os.path.join(path_to_dataset_dir, '*.jpg'))
        total_files = len(path_to_image_files)
        print ('%d files found in %s' % (total_files, path_to_dataset_dir))

#         with h5py.File(path_to_digit_struct_mat_file, 'r') as digit_struct_mat_file:
        example_reader = ExampleReader(path_to_image_files)

        #read in csv
        csv_path = "/home/egert/Desktop/Number_reader_from_sportfoto/images_data.csv"
        data = pd.read_csv(csv_path, index_col='image')
        ###

        for index, path_to_image_file in enumerate(path_to_image_files):
            print ('(%d/%d) processing %s' % (index + 1, total_files, path_to_image_file))

#                 example = example_reader.read_and_convert(digit_struct_mat_file)
            example = example_reader.read_and_convert_sportfoto(data)

            if example is None:
                break

            idx = choose_writer_callback(path_to_tfrecords_files)
            writers[idx].write(example.SerializeToString())
            num_examples[idx] += 1

    for writer in writers:
        writer.close()

    return num_examples


def create_tfrecords_meta_file(num_train_examples, num_val_examples, num_test_examples,
                               path_to_tfrecords_meta_file):
    print ('Saving meta file to %s...' % path_to_tfrecords_meta_file)
#    meta = Meta()
#    meta.num_train_examples = num_train_examples
#    meta.num_val_examples = num_val_examples
#    meta.num_test_examples = num_test_examples
#    meta.save(path_to_tfrecords_meta_file)
    
    with open(path_to_tfrecords_meta_file, 'w') as f:
            content = {
                'num_examples': {
                    'train': num_train_examples,
                    'val': num_val_examples,
                    'test': num_test_examples
                }
            }
            json.dump(content, f)
            
    

# # def main(_):
# path_to_train_dir = './data/train'
# path_to_test_dir = './data/test'
# path_to_extra_dir = './data/extra'
# path_to_train_digit_struct_mat_file = './data/train/digitStruct.mat'
# path_to_test_digit_struct_mat_file = './data/test/digitStruct.mat'
# path_to_extra_digit_struct_mat_file = './data/extra/digitStruct.mat'

# path_to_train_tfrecords_file = os.path.join('./data', 'train.tfrecords')
# path_to_val_tfrecords_file = os.path.join('./data', 'val.tfrecords')
# path_to_test_tfrecords_file = os.path.join('./data', 'test.tfrecords')
# path_to_tfrecords_meta_file = os.path.join('./data', 'meta.json')


# def main(_):
path_to_train_dir = './toy_data/train'
path_to_test_dir = './toy_data/test'
path_to_extra_dir = './toy_data/extra'
path_to_train_digit_struct_mat_file = './toy_data/train/digitStruct.mat'
path_to_test_digit_struct_mat_file = './toy_data/test/digitStruct.mat'
path_to_extra_digit_struct_mat_file = './toy_data/extra/digitStruct.mat'

path_to_train_tfrecords_file = os.path.join('./toy_data', 'train.tfrecords')
path_to_val_tfrecords_file = os.path.join('./toy_data', 'val.tfrecords')
path_to_test_tfrecords_file = os.path.join('./toy_data', 'test.tfrecords')
path_to_tfrecords_meta_file = os.path.join('./toy_data', 'meta.json')


for path_to_file in [path_to_train_tfrecords_file, path_to_val_tfrecords_file, path_to_test_tfrecords_file]:
    assert not os.path.exists(path_to_file), 'The file %s already exists' % path_to_file

print ('Processing training and validation data...')
[num_train_examples, num_val_examples] = convert_to_tfrecords([(path_to_train_dir, path_to_train_digit_struct_mat_file),
                                                                (path_to_extra_dir, path_to_extra_digit_struct_mat_file)],
                                                                [path_to_train_tfrecords_file, path_to_val_tfrecords_file],
                                                                lambda paths: 0 if random.random() > 0.1 else 1)
print ('Processing test data...')
[num_test_examples] = convert_to_tfrecords([(path_to_test_dir, path_to_test_digit_struct_mat_file)],
                                            [path_to_test_tfrecords_file],
                                            lambda paths: 0)

create_tfrecords_meta_file(num_train_examples, num_val_examples, num_test_examples,
                            path_to_tfrecords_meta_file)

print ('Done')


# if __name__ == '__main__':
#     tf.app.run(main=main)



Processing training and validation data...
5088 files found in ./toy_data/train
(1/5088) processing ./toy_data/train/3922544.jpg
(2/5088) processing ./toy_data/train/3928534.jpg
(3/5088) processing ./toy_data/train/3863985.jpg
(4/5088) processing ./toy_data/train/3881133.jpg
(5/5088) processing ./toy_data/train/3915638.jpg
(6/5088) processing ./toy_data/train/3920200.jpg
(7/5088) processing ./toy_data/train/3878113.jpg
(8/5088) processing ./toy_data/train/3905061.jpg
(9/5088) processing ./toy_data/train/3894333.jpg
(10/5088) processing ./toy_data/train/3894243.jpg
(11/5088) processing ./toy_data/train/3862765.jpg
(12/5088) processing ./toy_data/train/3918896.jpg
(13/5088) processing ./toy_data/train/3889259.jpg
(14/5088) processing ./toy_data/train/3910033.jpg
(15/5088) processing ./toy_data/train/3910782.jpg
(16/5088) processing ./toy_data/train/3926693.jpg
(17/5088) processing ./toy_data/train/3917323.jpg
(18/5088) processing ./toy_data/train/3900983.jpg
(19/5088) processing ./toy_da

(178/5088) processing ./toy_data/train/3893113.jpg
(179/5088) processing ./toy_data/train/3894215.jpg
(180/5088) processing ./toy_data/train/3918786.jpg
(181/5088) processing ./toy_data/train/3894807.jpg
(182/5088) processing ./toy_data/train/3900321.jpg
(183/5088) processing ./toy_data/train/3875854.jpg
(184/5088) processing ./toy_data/train/3881296.jpg
(185/5088) processing ./toy_data/train/3907033.jpg
(186/5088) processing ./toy_data/train/3883993.jpg
(187/5088) processing ./toy_data/train/3873169.jpg
(188/5088) processing ./toy_data/train/3875176.jpg
(189/5088) processing ./toy_data/train/3917413.jpg
(190/5088) processing ./toy_data/train/3883441.jpg
(191/5088) processing ./toy_data/train/3896733.jpg
(192/5088) processing ./toy_data/train/3907189.jpg
(193/5088) processing ./toy_data/train/3916867.jpg
(194/5088) processing ./toy_data/train/3881259.jpg
(195/5088) processing ./toy_data/train/3905000.jpg
(196/5088) processing ./toy_data/train/3898415.jpg
(197/5088) processing ./toy_dat

(343/5088) processing ./toy_data/train/3869945.jpg
(344/5088) processing ./toy_data/train/3921856.jpg
(345/5088) processing ./toy_data/train/3930380.jpg
(346/5088) processing ./toy_data/train/3873773.jpg
(347/5088) processing ./toy_data/train/3895015.jpg
(348/5088) processing ./toy_data/train/3901563.jpg
(349/5088) processing ./toy_data/train/3917471.jpg
(350/5088) processing ./toy_data/train/3890721.jpg
(351/5088) processing ./toy_data/train/3894819.jpg
(352/5088) processing ./toy_data/train/3905200.jpg
(353/5088) processing ./toy_data/train/3919419.jpg
(354/5088) processing ./toy_data/train/3877389.jpg
(355/5088) processing ./toy_data/train/3920202.jpg
(356/5088) processing ./toy_data/train/3901814.jpg
(357/5088) processing ./toy_data/train/3905232.jpg
(358/5088) processing ./toy_data/train/3930185.jpg
(359/5088) processing ./toy_data/train/3918047.jpg
(360/5088) processing ./toy_data/train/3863996.jpg
(361/5088) processing ./toy_data/train/3915704.jpg
(362/5088) processing ./toy_dat

(514/5088) processing ./toy_data/train/3886173.jpg
(515/5088) processing ./toy_data/train/3926459.jpg
(516/5088) processing ./toy_data/train/3902319.jpg
(517/5088) processing ./toy_data/train/3930646.jpg
(518/5088) processing ./toy_data/train/3916617.jpg
(519/5088) processing ./toy_data/train/3875049.jpg
(520/5088) processing ./toy_data/train/3918920.jpg
(521/5088) processing ./toy_data/train/3878387.jpg
(522/5088) processing ./toy_data/train/3888045.jpg
(523/5088) processing ./toy_data/train/3905783.jpg
(524/5088) processing ./toy_data/train/3900909.jpg
(525/5088) processing ./toy_data/train/3887552.jpg
(526/5088) processing ./toy_data/train/3926576.jpg
(527/5088) processing ./toy_data/train/3925902.jpg
(528/5088) processing ./toy_data/train/3906535.jpg
(529/5088) processing ./toy_data/train/3924675.jpg
(530/5088) processing ./toy_data/train/3893563.jpg
(531/5088) processing ./toy_data/train/3864737.jpg
(532/5088) processing ./toy_data/train/3897443.jpg
(533/5088) processing ./toy_dat

(675/5088) processing ./toy_data/train/3881959.jpg
(676/5088) processing ./toy_data/train/3873946.jpg
(677/5088) processing ./toy_data/train/3921886.jpg
(678/5088) processing ./toy_data/train/3898447.jpg
(679/5088) processing ./toy_data/train/3902695.jpg
(680/5088) processing ./toy_data/train/3919762.jpg
(681/5088) processing ./toy_data/train/3895297.jpg
(682/5088) processing ./toy_data/train/3883535.jpg
(683/5088) processing ./toy_data/train/3874235.jpg
(684/5088) processing ./toy_data/train/3919010.jpg
(685/5088) processing ./toy_data/train/3926549.jpg
(686/5088) processing ./toy_data/train/3910510.jpg
(687/5088) processing ./toy_data/train/3893682.jpg
(688/5088) processing ./toy_data/train/3874507.jpg
(689/5088) processing ./toy_data/train/3893556.jpg
(690/5088) processing ./toy_data/train/3890958.jpg
(691/5088) processing ./toy_data/train/3874602.jpg
(692/5088) processing ./toy_data/train/3918577.jpg
(693/5088) processing ./toy_data/train/3862559.jpg
(694/5088) processing ./toy_dat

...

(468/954) processing ./toy_data/extra/3866981.jpg
(469/954) processing ./toy_data/extra/3873251.jpg
(470/954) processing ./toy_data/extra/3920998.jpg
(471/954) processing ./toy_data/extra/3906492.jpg
(472/954) processing ./toy_data/extra/3917734.jpg
(473/954) processing ./toy_data/extra/3905162.jpg
(474/954) processing ./toy_data/extra/3863902.jpg
(475/954) processing ./toy_data/extra/3930918.jpg
(476/954) processing ./toy_data/extra/3883321.jpg
(477/954) processing ./toy_data/extra/3872689.jpg
(478/954) processing ./toy_data/extra/3869013.jpg
(479/954) processing ./toy_data/extra/3900197.jpg
(480/954) processing ./toy_data/extra/3896233.jpg
(481/954) processing ./toy_data/extra/3922070.jpg
(482/954) processing ./toy_data/extra/3862507.jpg
(483/954) processing ./toy_data/extra/3917649.jpg
(484/954) processing ./toy_data/extra/3865449.jpg
(485/954) processing ./toy_data/extra/3902753.jpg
(486/954) processing ./toy_data/extra/3918307.jpg
(487/954) processing ./toy_data/extra/3930123.jpg


(638/954) processing ./toy_data/extra/3867035.jpg
(639/954) processing ./toy_data/extra/3919042.jpg
(640/954) processing ./toy_data/extra/3896418.jpg
(641/954) processing ./toy_data/extra/3897410.jpg
(642/954) processing ./toy_data/extra/3881371.jpg
(643/954) processing ./toy_data/extra/3898557.jpg
(644/954) processing ./toy_data/extra/3904835.jpg
(645/954) processing ./toy_data/extra/3918856.jpg
(646/954) processing ./toy_data/extra/3895580.jpg
(647/954) processing ./toy_data/extra/3919493.jpg
(648/954) processing ./toy_data/extra/3922766.jpg
(649/954) processing ./toy_data/extra/3924233.jpg
(650/954) processing ./toy_data/extra/3876579.jpg
(651/954) processing ./toy_data/extra/3900869.jpg
(652/954) processing ./toy_data/extra/3887087.jpg
(653/954) processing ./toy_data/extra/3902253.jpg
(654/954) processing ./toy_data/extra/3923046.jpg
(655/954) processing ./toy_data/extra/3904096.jpg
(656/954) processing ./toy_data/extra/3894659.jpg
(657/954) processing ./toy_data/extra/3905296.jpg


(809/954) processing ./toy_data/extra/3877757.jpg
(810/954) processing ./toy_data/extra/3922874.jpg
(811/954) processing ./toy_data/extra/3875862.jpg
(812/954) processing ./toy_data/extra/3915788.jpg
(813/954) processing ./toy_data/extra/3893800.jpg
(814/954) processing ./toy_data/extra/3899373.jpg
(815/954) processing ./toy_data/extra/3867509.jpg
(816/954) processing ./toy_data/extra/3898595.jpg
(817/954) processing ./toy_data/extra/3874577.jpg
(818/954) processing ./toy_data/extra/3905055.jpg
(819/954) processing ./toy_data/extra/3874016.jpg
(820/954) processing ./toy_data/extra/3916761.jpg
(821/954) processing ./toy_data/extra/3920889.jpg
(822/954) processing ./toy_data/extra/3924765.jpg
(823/954) processing ./toy_data/extra/3894224.jpg
(824/954) processing ./toy_data/extra/3919954.jpg
(825/954) processing ./toy_data/extra/3875764.jpg
(826/954) processing ./toy_data/extra/3925979.jpg
(827/954) processing ./toy_data/extra/3869082.jpg
(828/954) processing ./toy_data/extra/3886307.jpg


(24/318) processing ./toy_data/test/3929191.jpg
(25/318) processing ./toy_data/test/3881192.jpg
(26/318) processing ./toy_data/test/3905522.jpg
(27/318) processing ./toy_data/test/3881593.jpg
(28/318) processing ./toy_data/test/3923049.jpg
(29/318) processing ./toy_data/test/3902589.jpg
(30/318) processing ./toy_data/test/3923603.jpg
(31/318) processing ./toy_data/test/3864142.jpg
(32/318) processing ./toy_data/test/3875575.jpg
(33/318) processing ./toy_data/test/3881904.jpg
(34/318) processing ./toy_data/test/3921934.jpg
(35/318) processing ./toy_data/test/3926554.jpg
(36/318) processing ./toy_data/test/3891762.jpg
(37/318) processing ./toy_data/test/3896927.jpg
(38/318) processing ./toy_data/test/3887058.jpg
(39/318) processing ./toy_data/test/3923036.jpg
(40/318) processing ./toy_data/test/3864481.jpg
(41/318) processing ./toy_data/test/3862560.jpg
(42/318) processing ./toy_data/test/3887646.jpg
(43/318) processing ./toy_data/test/3875511.jpg
(44/318) processing ./toy_data/test/3905

(195/318) processing ./toy_data/test/3906946.jpg
(196/318) processing ./toy_data/test/3919939.jpg
(197/318) processing ./toy_data/test/3900858.jpg
(198/318) processing ./toy_data/test/3886916.jpg
(199/318) processing ./toy_data/test/3918449.jpg
(200/318) processing ./toy_data/test/3897502.jpg
(201/318) processing ./toy_data/test/3904876.jpg
(202/318) processing ./toy_data/test/3921487.jpg
(203/318) processing ./toy_data/test/3878210.jpg
(204/318) processing ./toy_data/test/3887813.jpg
(205/318) processing ./toy_data/test/3885665.jpg
(206/318) processing ./toy_data/test/3898630.jpg
(207/318) processing ./toy_data/test/3915286.jpg
(208/318) processing ./toy_data/test/3881673.jpg
(209/318) processing ./toy_data/test/3880292.jpg
(210/318) processing ./toy_data/test/3916889.jpg
(211/318) processing ./toy_data/test/3904741.jpg
(212/318) processing ./toy_data/test/3881273.jpg
(213/318) processing ./toy_data/test/3867346.jpg
(214/318) processing ./toy_data/test/3878536.jpg
(215/318) processing